In [64]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/7j/c3cwb7d17c1cfkhmkrqtw4q00000gn/T/ipykernel_10450/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [65]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import os
print(os.getcwd())
from nltk.stem import WordNetLemmatizer



/Users/bshanmugam/Documents/AIENginnering/AIClasswork/lab-natural-language-processing


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [66]:
## Read Data for the Fraudulent Email Kaggle Challenge
import os
os.chdir('/Users/bshanmugam/Documents/AIENginnering/AIClasswork/lab-natural-language-processing')

data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)

print(data.shape)
data.fillna("",inplace=True)
print(data.columns)


(1000, 2)
Index(['text', 'label'], dtype='object')


### Let's divide the training and test set into two partitions

In [75]:
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

## Data Preprocessing

In [68]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [69]:
import re
def clean_html(text):
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL | re.IGNORECASE)
     # Remove HTML comments <!-- ... -->
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [70]:
import re

def clean_text(text):
    # Remove prefixed 'b' (like from byte strings)
    text = re.sub(r"^b['\"]", '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove special characters (keep only letters and spaces)
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Remove all single characters surrounded by spaces
    text = re.sub(r'\s+[a-z]\s+', ' ', text)
    
    # Remove single characters at the start
    text = re.sub(r'^[a-z]\s+', '', text)
    
    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    
    # Strip leading and trailing spaces
    text = text.strip()
    
    return text

data['cleaned_html'] = data['text'].apply(clean_html)  
data['final_cleaned'] = data['cleaned_html'].apply(clean_text)  


## Now let's work on removing stopwords
Remove the stopwords.

In [71]:
def remove_stopwords_and_stem(text):
    tokens = text.split()  # split text into words
    filtered = [snowball.stem(word) for word in tokens if word.lower() not in stopwords.words('english')]
    return " ".join(filtered)
data['clean_no_stop_stem'] = data['final_cleaned'].apply(remove_stopwords_and_stem)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [72]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def tame_text(text):
    tokens = text.lower().split()
    cleaned_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and token.isalpha()]
    
    # Join back to a single string
    return " ".join(cleaned_tokens)

data['lemmatization_word']= data['final_cleaned'].apply(tame_text)

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [73]:
# Your code
corpus = data['lemmatization_word'].tolist()
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(corpus)
word_counts = X_bow.toarray().sum(axis=0)
words = vectorizer.get_feature_names_out()
word_freq = pd.DataFrame({'word': words, 'count': word_counts})
top10_words = word_freq.sort_values(by='count', ascending=False).head(10)
print(top10_words)

              word  count
10953        money    924
167        account    800
1904          bank    754
7118          fund    704
2724      business    496
4191       country    457
17535  transaction    416
10765      million    396
17606     transfer    393
3629       company    386


## Extra features

In [77]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['lemmatization_word'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['lemmatization_word'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['lemmatization_word'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['lemmatization_word'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['lemmatization_word'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['lemmatization_word'].apply(lambda x: len(x)) 

data_train.head()

,text,label,cleaned_html,final_cleaned,clean_no_stop_stem,lemmatization_word,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",regards mr nelson smithkindly reply me on my p...,regard mr nelson smithkind repli privat email ...,regard mr nelson smithkindly reply private ema...,1,0,75
535,I have not been able to reach oscar this am. W...,0,I have not been able to reach oscar this am. W...,have not been able to reach oscar this am we a...,abl reach oscar suppos send pdb receiv,able reach oscar supposed send pdb receive,1,0,42
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,; Huma Abedin B6I'm checking with Pat on the 5...,huma abedin bim checking with pat on the will ...,huma abedin bim check pat work jack jake resta...,huma abedin bim checking pat work jack jake re...,1,0,79
557,I can have it announced here on Monday - can't...,0,I can have it announced here on Monday - can't...,can have it announced here on monday cant today,announc monday cant today,announced monday cant today,1,0,27
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...,bank of africaagence san pedro bp san pedro co...,bank africaag san pedro bp san pedro cote divo...,bank africaagence san pedro bp san pedro cote ...,1,1,1067


## How would work the Bag of Words with Count Vectorizer concept?

In [79]:
from sklearn.feature_extraction.text import CountVectorizer


data['lemmatization_word'] = data['lemmatization_word'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)


vectorizer = CountVectorizer()

# Fit and transform your preprocessed text to get the Bag of Words matrix
X_bow = vectorizer.fit_transform(data['lemmatization_word'])

# Check the feature names (vocabulary words)
print("Vocabulary words:", vectorizer.get_feature_names_out())

# Convert the sparse matrix to array (optional, for visualization)
bow_array = X_bow.toarray()
print("Bag of Words matrix shape:", bow_array.shape)

# Example: Display Bag of Words for first 3 documents
print(bow_array[:10])


Vocabulary words: ['aac' 'aaclocated' 'aae' ... 'zumadirector' 'zumae' 'zurich']
Bag of Words matrix shape: (1000, 19352)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Make sure the column is string type (if it's a list, convert it to string)
data['lemmatization_word'] = data['lemmatization_word'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform your preprocessed text to get the TF-IDF matrix
X_tfidf = tfidf_vectorizer.fit_transform(data['lemmatization_word'])

# Check the feature names (vocabulary words)
print("Vocabulary words:", tfidf_vectorizer.get_feature_names_out())

# Convert sparse matrix to array (optional, for viewing)
tfidf_array = X_tfidf.toarray()
print("TF-IDF matrix shape:", tfidf_array.shape)

# Example: Display TF-IDF vectors for first 3 documents
print(tfidf_array[:3])


Vocabulary words: ['aac' 'aaclocated' 'aae' ... 'zumadirector' 'zumae' 'zurich']
TF-IDF matrix shape: (1000, 19352)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## And the Train a Classifier?

In [81]:
# Your codefrom sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assume X = feature matrix (e.g., X_tfidf), y = labels (like data['label'])
X = X_tfidf  # or X_bow if you prefer Bag of Words
y = data['label']  # Your target column

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the classifier
clf = LogisticRegression(max_iter=1000)

# Train the classifier on training data
clf.fit(X_train, y_train)

# Predict on the test data
y_pred = clf.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


NameError: name 'LogisticRegression' is not defined

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code